In [1]:
%conda install plotly -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.11.0
  latest version: 24.1.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import os
import sys
import pathlib
import open3d as o3d
import plotly
import plotly.graph_objects as go
import exifread

In [3]:
def get_colors_from_ply(plyfile):
    '''
    Function to read the colors from a .ply file
    returns an RGB array scaled between [0,1]
    '''
    with open(plyfile, 'r') as f:
        colors = []
        columns = {}
        col_idx = 0
        header_done = False
        for line in f.readlines():
            if line.startswith('property'):
                columns[line.split()[-1]] = col_idx
                col_idx += 1
            if header_done:
                line_list = line.split()
                colors.append([float(line_list[columns['diffuse_red']]),
                              float(line_list[columns['diffuse_green']]),
                              float(line_list[columns['diffuse_blue']])
                              ])
            if line.startswith('end_header'):
                header_done = True
        colors = np.array(colors)/255
    return colors

In [4]:
# read point cloud data
pcd = o3d.io.read_point_cloud("mickey_with_ears/merged.ply", format='ply')
# store the colors
pcd.colors = o3d.utility.Vector3dVector(get_colors_from_ply('mickey_with_ears/merged.ply'))

# convert to array
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=points[:,0], y=points[:,1], z=points[:,2], 
            mode='markers',
            marker=dict(size=5, color=colors)
        )
    ],
    layout=dict(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )
)
fig.write_html('vis-dense.html')